In [1]:
!pip install -q transformers accelerate torch torchvision timm bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.9 MB/s eta 0:00:00


In [5]:
# Install huggingface_hub if not already installed (it's usually installed with transformers)
!pip install -q huggingface_hub

# Log in to Hugging Face Hub
# You will be prompted to enter your Hugging Face token.
# Generate one in your Hugging Face account settings -> Access Tokens
!huggingface-cli login

# --- 🩺 Wound Analyzer AI Assistant ---
# ✅ Uses BLIP-2 to describe the wound from an image
# ✅ Uses a Medical LLM (MedAlpaca or similar) to provide first aid advice
# ✅ Free and open-source setup

# 📦 Step 1: Install dependencies
!pip install -q transformers accelerate torch torchvision timm bitsandbytes


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `smol agent` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authentica

In [6]:
# --- 🩺 Wound Analyzer AI Assistant ---
# ✅ Uses BLIP-2 to describe the wound from an image
# ✅ Uses a Medical LLM (MedAlpaca or similar) to provide first aid advice
# ✅ Free and open-source setup

# 📦 Step 1: Install dependencies


# 📸 Step 2: Load and process the image using BLIP-2
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Load BLIP-2 (Vision-Language Model)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    device_map="auto",
    torch_dtype=torch.float16
)

# Upload wound image
from google.colab import files
uploaded = files.upload()

# Load uploaded image
image_path = list(uploaded.keys())[0]
raw_image = Image.open(image_path).convert('RGB')

# Describe the wound
prompt = "Describe the wound and suggest the type of injury."
inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to("cuda", torch.float16)
out = model.generate(**inputs, max_new_tokens=100)
caption = processor.decode(out[0], skip_special_tokens=True)
print("\n Wound Description: ", caption)

# 💬 Step 3: Ask a medical LLM about treatment
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load a light medical LLM (or use a general small LLM)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"  # You can replace with MedAlpaca if available

tokenizer = AutoTokenizer.from_pretrained(model_name)
med_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

# Ask treatment based on the caption
query = f"This wound is described as: '{caption}'. What is the first aid treatment for it?"
inputs = tokenizer(query, return_tensors="pt").to("cuda")
output = med_model.generate(**inputs, max_new_tokens=200)
answer = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n First Aid Advice: ", answer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Saving WhatsApp Image 2025-06-12 at 11.22.08 AM.jpeg to WhatsApp Image 2025-06-12 at 11.22.08 AM (3).jpeg

🔍 Wound Description:  Describe the wound and suggest the type of injury.



OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
403 Client Error. (Request ID: Root=1-684a7083-64ab34956be3b8e26d703e46;cd524d87-71ba-4bd7-9195-b22adcf90007)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 to ask for access.

In [8]:

# 📸 Step 2: Load and process the image using BLIP-2
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Load BLIP-2 (Vision-Language Model)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    device_map="auto",
    torch_dtype=torch.float16
)

# Upload wound image
from google.colab import files
uploaded = files.upload()

# Load uploaded image
image_path = list(uploaded.keys())[0]
raw_image = Image.open(image_path).convert('RGB')

# Describe the wound
prompt = "Describe the wound and suggest the type of injury."
inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to("cuda", torch.float16)
out = model.generate(**inputs, max_new_tokens=100)
caption = processor.decode(out[0], skip_special_tokens=True)
print("\n Wound Description: ", caption)

# Step 3: Ask Falcon-7B-Instruct for treatment advice
from transformers import AutoTokenizer, AutoModelForCausalLM

# Use Falcon-7B-Instruct (Open and free)
model_name = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
med_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# Ask treatment based on the caption
query = f"This wound is described as: '{caption}'. What is the first aid treatment for it?"
inputs = tokenizer(query, return_tensors="pt").to("cuda")
output = med_model.generate(**inputs, max_new_tokens=200)
answer = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n First Aid Advice: ", answer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Saving WhatsApp Image 2025-06-12 at 11.22.08 AM.jpeg to WhatsApp Image 2025-06-12 at 11.22.08 AM (5).jpeg

🔍 Wound Description:  Describe the wound and suggest the type of injury.



tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 564.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 70.12 MiB is free. Process 2638 has 14.67 GiB memory in use. Of the allocated memory 13.91 GiB is allocated by PyTorch, and 652.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)